In [120]:
#import sample data: adult income for classification
import pandas as pd
import numpy as np

df = pd.read_csv('./data/adult/adult.csv', header=0, \
                dtype={'Workclass':object,
                      'Education':object,
                      'Marital_Status':object,
                      'Occupation':object,
                      'Relationship':object,
                      'Race':object,
                      'Sex':object,
                      'Native_country':object,
                      'Income':object})
X = df.iloc[:,:-1]
y = df.iloc[:,[-1]]

#check the shape
print('----------------------------------------------------------------------------------------')
print('Raw shape: (%i,%i)' %df.shape)
print('X shape: (%i,%i)' %X.shape)
print('y shape: (%i,%i)' %y.shape)
print('----------------------------------------------------------------------------------------')
print(X.dtypes)
print('----------------------------------------------------------------------------------------')
print('Check the null count of the target variable: %i' % y.isnull().sum())
print('----------------------------------------------------------------------------------------')
#収入が50K$を超える(>50K)を1(正例)として変換
class_mapping = {'>50K':1,'<=50K':0}
y_new = y.copy()
y_new.loc[:,'Income'] = y_new['Income'].map(class_mapping)
print(y_new.groupby(['Income']).size())
X.head(15)
#ちょっとしたNaNの確認
print('----------------------------------------------------------------------------------------')
print(X.iloc[:,[13]].isnull().sum())

----------------------------------------------------------------------------------------
Raw shape: (32561,15)
X shape: (32561,14)
y shape: (32561,1)
----------------------------------------------------------------------------------------
Age                int64
Workclass         object
Fnlwgt             int64
Education         object
Education_num      int64
Marital_Status    object
Occupation        object
Relationship      object
Race              object
Sex               object
Capital_Gain       int64
Capital_Loss       int64
Hours_Per_Week     int64
Native_Country    object
dtype: object
----------------------------------------------------------------------------------------
Check the null count of the target variable: 0
----------------------------------------------------------------------------------------
Income
0    24720
1     7841
dtype: int64
----------------------------------------------------------------------------------------
Native_Country    583
dtype: int64


In [121]:
#データに含まれる欠損値はNaNではなく'?'であり、連続値に含まれているかを調べる。
#all_columns = list(X.columns)
#print(all_columns)
#print('----------------------------------------------------------------------------------------')
#print(df.groupby('Age').count())
#print(df[df['Age'] == 'NaN'].loc[:,'Age'])
#for column in all_columns:
#    count = df[df['Age'] == '?'].loc[:,'Age'].count()
#    print('Check the ? count of %s : %d') #%column, %count )

#連続値の中に含まれる?を数えようと思ったんだけど無理でした。

In [122]:
ohe_columns = ['Workclass','Education','Marital_Status','Occupation','Relationship','Race','Sex','Native_Country']
X_ohe = pd.get_dummies(X, dummy_na=True, columns=ohe_columns)
print('X_ohe shape:(%i,%i)' %X_ohe.shape)

X_ohe shape:(32561,113)


In [123]:
from sklearn.preprocessing import Imputer

#欠損値NaNを平均値(mean)で置換
imp = Imputer(missing_values='NaN',strategy='mean',axis=0)
imp.fit(X_ohe)

#学習済みImputerを適用し、X_newの欠損値を置換
X_ohe_columns = X_ohe.columns.values
X_ohe = pd.DataFrame(imp.transform(X_ohe), columns=X_ohe_columns)

#結果表示
X_ohe.head()

,Age,Fnlwgt,Education_num,Capital_Gain,Capital_Loss,Hours_Per_Week,Workclass_Federal-gov,Workclass_Local-gov,Workclass_Never-worked,Workclass_Private,...,Native_Country_Puerto-Rico,Native_Country_Scotland,Native_Country_South,Native_Country_Taiwan,Native_Country_Thailand,Native_Country_Trinadad&Tobago,Native_Country_United-States,Native_Country_Vietnam,Native_Country_Yugoslavia,Native_Country_nan
0,39.0,77516.0,13.0,2174.0,0.0,40.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,50.0,83311.0,13.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,38.0,215646.0,9.0,0.0,0.0,40.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,53.0,234721.0,7.0,0.0,0.0,40.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,28.0,338409.0,13.0,0.0,0.0,40.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [248]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingClassifier

selector = RFE(GradientBoostingClassifier(random_state=1), n_features_to_select=15, step=.05)
selector.fit(X_ohe, y.as_matrix().ravel())

X_fin = X_ohe.loc[:, X_ohe_columns[selector.support_]]
print('X_fin shape:(%i,%i)' % X_fin.shape)
X_fin.head()

X_fin shape:(32561,15)


,Age,Fnlwgt,Education_num,Capital_Gain,Capital_Loss,Hours_Per_Week,Workclass_Self-emp-not-inc,Marital_Status_Married-civ-spouse,Occupation_Exec-managerial,Occupation_Farming-fishing,Occupation_Other-service,Occupation_Prof-specialty,Occupation_Tech-support,Relationship_Wife,Sex_Male
0,39.0,77516.0,13.0,2174.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,50.0,83311.0,13.0,0.0,0.0,13.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,38.0,215646.0,9.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,53.0,234721.0,7.0,0.0,0.0,40.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,28.0,338409.0,13.0,0.0,0.0,40.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [249]:
import pandas as pd

# import sample data
# Loan screening data for classification 
df_s = pd.read_csv('./data/adult/adult_test.csv', header=0, \
                dtype={'Workclass':object,
                      'Education':object,
                      'Marital_Status':object,
                      'Occupation':object,
                      'Relationship':object,
                      'Race':object,
                      'Sex':object,
                      'Native_country':object,
                      'Income':object})
X_s = df_s.iloc[:,:-1]
y_s = df_s.iloc[:,[-1]]

print('----------------------------------------------------------------------------------------')
print('Raw shape: (%i,%i)' %df_s.shape)
print('X_s shape: (%i,%i)' %X_s.shape)
print('y_s shape: (%i,%i)' %y_s.shape)
print(X_s.dtypes)

----------------------------------------------------------------------------------------
Raw shape: (16281,15)
X_s shape: (16281,14)
y_s shape: (16281,1)
Age                int64
Workclass         object
Fnlwgt             int64
Education         object
Education_num      int64
Marital_Status    object
Occupation        object
Relationship      object
Race              object
Sex               object
Capital_Gain       int64
Capital_Loss       int64
Hours_Per_Week     int64
Native_Country    object
dtype: object


In [250]:
ohe_columns = ['Workclass','Education','Marital_Status','Occupation','Relationship','Race','Sex','Native_Country']
X_ohe_s = pd.get_dummies(X_s, dummy_na=True, columns=ohe_columns)
print('X_ohe_s shape:(%i,%i)' % X_ohe_s.shape)
X_ohe_s.head()

X_ohe_s shape:(16281,112)


,Age,Fnlwgt,Education_num,Capital_Gain,Capital_Loss,Hours_Per_Week,Workclass_Federal-gov,Workclass_Local-gov,Workclass_Never-worked,Workclass_Private,...,Native_Country_Puerto-Rico,Native_Country_Scotland,Native_Country_South,Native_Country_Taiwan,Native_Country_Thailand,Native_Country_Trinadad&Tobago,Native_Country_United-States,Native_Country_Vietnam,Native_Country_Yugoslavia,Native_Country_nan
0,25,226802,7,0,0,40,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,38,89814,9,0,0,50,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,28,336951,12,0,0,40,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,44,160323,10,7688,0,40,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,18,103497,10,0,0,30,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [251]:
cols_model = set(X_ohe.columns.values)
cols_score = set(X_ohe_s.columns.values)

#モデルにはあったがスコアにはないデータ項目
diff1 = cols_model - cols_score
print('モデルのみに存在する項目: %s' % diff1)

diff2 = cols_score - cols_model
print('スコアのみに存在する項目: %s' % diff2)

モデルのみに存在する項目: {'Native_Country_Holand-Netherlands'}
スコアのみに存在する項目: set()


モデルのみに一個あるので、スコアに付け足す

In [252]:
df_cols_m = pd.DataFrame(None, columns=X_ohe_columns, dtype=float)
df_cols_m

,Age,Fnlwgt,Education_num,Capital_Gain,Capital_Loss,Hours_Per_Week,Workclass_Federal-gov,Workclass_Local-gov,Workclass_Never-worked,Workclass_Private,...,Native_Country_Puerto-Rico,Native_Country_Scotland,Native_Country_South,Native_Country_Taiwan,Native_Country_Thailand,Native_Country_Trinadad&Tobago,Native_Country_United-States,Native_Country_Vietnam,Native_Country_Yugoslavia,Native_Country_nan


In [253]:
X_ohe_s2 = pd.concat([df_cols_m, X_ohe_s])
print(X_ohe_s2.loc[:,'Native_Country_Holand-Netherlands'].head())
X_ohe_s2.head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Native_Country_Holand-Netherlands, dtype: float64


,Age,Capital_Gain,Capital_Loss,Education_10th,Education_11th,Education_12th,Education_1st-4th,Education_5th-6th,Education_7th-8th,Education_9th,...,Sex_nan,Workclass_Federal-gov,Workclass_Local-gov,Workclass_Never-worked,Workclass_Private,Workclass_Self-emp-inc,Workclass_Self-emp-not-inc,Workclass_State-gov,Workclass_Without-pay,Workclass_nan
0,25.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,44.0,7688.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


スコアリングでは登場しなかったデータ項目をゼロ埋めする

In [254]:
X_ohe_s2.loc[:,list(set(X_ohe.columns.values)-set(X_ohe_s.columns.values))] = \
    X_ohe_s2.loc[:,list(set(X_ohe.columns.values)-set(X_ohe_s.columns.values))].fillna(0,axis=1)
print(X_ohe_s2.loc[:,'Native_Country_Holand-Netherlands'].head())
X_ohe_s2.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Native_Country_Holand-Netherlands, dtype: float64


,Age,Capital_Gain,Capital_Loss,Education_10th,Education_11th,Education_12th,Education_1st-4th,Education_5th-6th,Education_7th-8th,Education_9th,...,Sex_nan,Workclass_Federal-gov,Workclass_Local-gov,Workclass_Never-worked,Workclass_Private,Workclass_Self-emp-inc,Workclass_Self-emp-not-inc,Workclass_State-gov,Workclass_Without-pay,Workclass_nan
0,25.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,44.0,7688.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


モデリング時点のデータ項目の並び順を明示的に担保する。

In [255]:
X_ohe_s2 = X_ohe_s2.reindex(X_ohe.columns.values, axis=1)
X_ohe_s2.head()

,Age,Fnlwgt,Education_num,Capital_Gain,Capital_Loss,Hours_Per_Week,Workclass_Federal-gov,Workclass_Local-gov,Workclass_Never-worked,Workclass_Private,...,Native_Country_Puerto-Rico,Native_Country_Scotland,Native_Country_South,Native_Country_Taiwan,Native_Country_Thailand,Native_Country_Trinadad&Tobago,Native_Country_United-States,Native_Country_Vietnam,Native_Country_Yugoslavia,Native_Country_nan
0,25.0,226802.0,7.0,0.0,0.0,40.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,38.0,89814.0,9.0,0.0,0.0,50.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,28.0,336951.0,12.0,0.0,0.0,40.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,44.0,160323.0,10.0,7688.0,0.0,40.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,18.0,103497.0,10.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [256]:
X_ohe_s3 = pd.DataFrame(imp.transform(X_ohe_s2), columns=X_ohe_columns)
X_ohe_s3.head()

,Age,Fnlwgt,Education_num,Capital_Gain,Capital_Loss,Hours_Per_Week,Workclass_Federal-gov,Workclass_Local-gov,Workclass_Never-worked,Workclass_Private,...,Native_Country_Puerto-Rico,Native_Country_Scotland,Native_Country_South,Native_Country_Taiwan,Native_Country_Thailand,Native_Country_Trinadad&Tobago,Native_Country_United-States,Native_Country_Vietnam,Native_Country_Yugoslavia,Native_Country_nan
0,25.0,226802.0,7.0,0.0,0.0,40.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,38.0,89814.0,9.0,0.0,0.0,50.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,28.0,336951.0,12.0,0.0,0.0,40.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,44.0,160323.0,10.0,7688.0,0.0,40.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,18.0,103497.0,10.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [257]:
X_fin_s = X_ohe_s3.loc[:, X_ohe_columns[selector.support_]]
print(X_fin_s.shape)
X_fin_s.head()

(16281, 15)


,Age,Fnlwgt,Education_num,Capital_Gain,Capital_Loss,Hours_Per_Week,Workclass_Self-emp-not-inc,Marital_Status_Married-civ-spouse,Occupation_Exec-managerial,Occupation_Farming-fishing,Occupation_Other-service,Occupation_Prof-specialty,Occupation_Tech-support,Relationship_Wife,Sex_Male
0,25.0,226802.0,7.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,38.0,89814.0,9.0,0.0,0.0,50.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,28.0,336951.0,12.0,0.0,0.0,40.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,44.0,160323.0,10.0,7688.0,0.0,40.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,18.0,103497.0,10.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [258]:
#check the shape
print('----------------------------------------------------------------------------------------')
print('X_fin shape:(%i,%i)' %X_fin.shape)
print('y_new shape:(%i,%i)' %y_new.shape)
print('X_fin_s shape:(%i,%i)' %X_fin_s.shape)
print('y_s shape:(%i,%i)' %y_s.shape)

----------------------------------------------------------------------------------------
X_fin shape:(32561,15)
y_new shape:(32561,1)
X_fin_s shape:(16281,15)
y_s shape:(16281,1)


In [259]:
# SET PARAMETER
SCORE = 'accuracy'

In [260]:
# import basice apis
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,r2_score

# Holdout
X_train,X_test,y_train,y_test = train_test_split(X_fin,y_new,test_size=0.20, random_state=1)

# set Pipelines
pipe_knn = Pipeline([('scl',StandardScaler()), ('est',KNeighborsClassifier())])
pipe_logistic = Pipeline([('scl', StandardScaler()), ('est',LogisticRegression(random_state=1))])
pipe_rf = Pipeline([('scl', StandardScaler()), ('est',RandomForestClassifier(random_state=1))])
pipe_gbc = Pipeline([('scl', StandardScaler()), ('est',GradientBoostingClassifier(random_state=1))])
pipe_mlpc = Pipeline([('scl', StandardScaler()), ('est',MLPClassifier(max_iter=500, random_state=1))])
pipe_svc = Pipeline([('scl', StandardScaler()), ('est',LinearSVC(random_state=1))])

# Evaluation function
def get_model_score(y_true, X_test, pipeline, score_type):
    if score_type == 'auc':
        # "1" stands for the Prob of positive samples for the binary classifcation
        # AUC requries the y_red would be score(probability). 
        y_pred = pipeline.predict_proba(X_test)[:,1]
        score = roc_auc_score(y_true, y_pred)
    elif score_type == 'f1':
        y_pred = pipeline.predict(X_test)
        score = f1_score(y_true, y_pred)
    elif score_type == 'precision':
        y_pred = pipeline.predict(X_test)
        score = precision_score(y_true, y_pred)
    elif score_type == 'accuracy':
        y_pred = pipeline.predict(X_test)
        score = accuracy_score(y_true, y_pred)
    elif score_type == 'recall':
        y_pred = pipeline.predict(X_test)
        score = recall_score(y_true, y_pred)      
    else:
        score = None
    return score

# Fit & Evaluation
pipe_names = ['pipe_knn', 'pipe_logistic', 'pipe_rf', 'pipe_gbc', 'pipe_mlpc', 'pipe_svc']
pipe_lines = [pipe_knn, pipe_logistic, pipe_rf, pipe_gbc, pipe_mlpc, pipe_svc]
model_score = []
for (i,pipe) in enumerate(pipe_lines):
    pipe.fit(X_train, y_train.as_matrix().ravel())
    score = get_model_score(y_test.as_matrix().ravel(), X_test, pipe, SCORE)
    print(i, ':', pipe_names[i],'-->',score)
    model_score.append(score)

0 : pipe_knn --> 0.8360202671579917
1 : pipe_logistic --> 0.8473821587594043
2 : pipe_rf --> 0.8453861507753724
3 : pipe_gbc --> 0.8667280822969445
4 : pipe_mlpc --> 0.8539843390142792
5 : pipe_svc --> 0.85014586212191


accuracy_scoreと、precision_scoreがよかった、pipe_gbcでモデルを構築する。recall_scoreはpipe_mlpcの方がよかったが、recall:再現率、取りこぼしがないよりもprecision:適合率、誤りがないを優先。

In [261]:
pipe_gbc.fit(X_fin, y_new.as_matrix().ravel())

predict_y = pd.DataFrame(pipe_gbc.predict(X_fin_s), columns=['Income_p'])
predict_y.head()

,Income_p
0,0
1,0
2,0
3,1
4,0


In [262]:
from sklearn.metrics import r2_score
#check the r2 score
y_new_s = y_s.copy()
class_mapping = {'>50K.':1,'<=50K.':0}
y_new_s.loc[:,'Income'] = y_new_s['Income'].map(class_mapping)
predict_y_a = predict_y.iloc[:16281,:]

#check the shape
print(y_new_s.groupby(['Income']).size())
print('----------------------------------------------------------------------------------------')
print('y_new shape : (%i,%i)' %y_new.shape)
print('y_new_s shape: (%i,%i)'%y_new_s.shape)
print('predict_y_a shape(%i,%i)'%predict_y_a.shape)
print('Check the null count of the target variable: %i' % y.isnull().sum())

#y_new_s.join(predict_y_a).head(30)
score_fin = accuracy_score(y_new_s, predict_y_a)
print('accuracy_score is %.6f' %score_fin)

Income
0    12435
1     3846
dtype: int64
----------------------------------------------------------------------------------------
y_new shape : (32561,1)
y_new_s shape: (16281,1)
predict_y_a shape(16281,1)
Check the null count of the target variable: 0
accuracy_score is 0.871568


accuracy_scoreが0.866961とまずまずのスコアで予想することができた。
ここから、どうやったらあげられるのか？
どうやったらもっと正確にできるのかを考えてみる

1.次元圧縮をものすごくしたが、もうちょっと次元圧縮を減らしてみる
2.fnlwgtなどのよくわからない数値を省いてみる。

などかな。他は今の所思いつかない

次元圧縮を項目10→15にしたら、ちょっと上がった
0.866961→0.871568になった。